###### this notebook shows my attempt (with the references from Kaggle) in understanding the concept of RAG and putting it in practice 
###### GenAI features used in this practice were by Google
###### VectorDB features provided by Chroma

In [ ]:
from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

In [ ]:
import os
from dotenv import load_dotenv
from google import genai

# Load environment variables from .env file
load_dotenv("/home/uih06736/ds/configs/.env")
key = os.getenv("GOOGLE_AI_STUDIO_API_KEY")

client = genai.Client(api_key=key)

# im taking a look at the list of available embeddings model available from the API
for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

In [ ]:
# creating some strings that represent documents

DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

##### CONCEPT: Embeddings
###### 1. Embeddings can represent text, images, and soon audio and video.
###### 2. They are in the form of vector(s), a numerical representations.
###### 3. These vector(s) can help in locating the relevant data points

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document" # this vectors is the best representation of the document
        else:
            embedding_task = "retrieval_query" # this vectors is the closest representation to the document's vectors

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

##### A vector database is used to store our vectors (embeddings)
##### these stored vectors will later be used to compare with the queried vectors(inference results)

In [ ]:
import chromadb

DB_NAME = "googlecardb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()

# transforming the string documents created earlier into DOCUMENT embeddings (the realest representation)
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

# inserting the embeddings of my artificial documents 
db.add(documents=documents, ids=[str(i) for i in range(len(documents))])
db.count()

##### An Embedding Model can create two different types of vectors—a Document Reference and a Query Probe—that are designed to align perfectly in the vector space, ensuring accurate and efficient retrieval.

In [ ]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "How do you use the touchscreen, just explain me to touchscreen"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

print(all_passages[0])

In [ ]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
"""

# Add the retrieved documents to the prompt.
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE: {passage_oneline}\n"

print(prompt)

In [ ]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

Markdown(answer.text)